# Mask Adhar Number (first 8)

## App working
- Downloaded AadharCard from Aadhar website(without Masked number)
- Start this App
- Decrypt PDF File : To Decrypt Encrypted PDF using password as caps first name 4 letter and year of birth
- PDF to IMG File : To convert decrypted Pdf file to Image file as png
- Auto Crop Aadhar IMG : To Crop Aadhar Card Size is given in default
- Mask Adhar Number : Main task To mask aadhar number first 8 number only
- Exit Application : To Exit Application.

In [8]:
# tkinter Application for Decrypt PDF file and Aadhar number Mask
import os
import re
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from tkinter.simpledialog import askstring
from pdf2image import convert_from_path, convert_from_bytes
from PyPDF2 import PdfFileWriter, PdfFileReader
path = r'C:\Program Files\poppler-20.11.0\bin'
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# tkinter
root= tk.Tk()
root.title("App: Decrypt+Img+Crop+Mask Aadhar number")
canvas = tk.Canvas(root, width = 300, height = 300, bg = 'lightsteelblue2', relief = 'raised')
canvas.pack()
label = tk.Label(root, text='Aadhar number Mask', bg = 'lightsteelblue2')
label.config(font=('helvetica', 20))
canvas.create_window(150, 30, window=label)
 
def decPdfFile():
#     out = PdfFileWriter() 
    file = filedialog.askopenfilename()
#     print(file)
    file = PdfFileReader(file) 
    # Check if the opened file is actually Encrypted 
    if file.isEncrypted:  
        password = askstring(title="Encrypted File" , prompt="Enter Password : ")
        if file.decrypt(password):
            out = PdfFileWriter()
            for idx in range(file.numPages): 
                page = file.getPage(idx) 
                out.addPage(page)
            saveAsFileName = askstring(title="File Save As " , prompt="Enter File Name : ")
            saveAsFileName = saveAsFileName + '.pdf'
            with open(saveAsFileName, "wb") as f:
                out.write(f) 
            messagebox.showinfo("PDF File", "File decrypted Successfully.")
        else:
            print("File decryption Failed...")
            messagebox.showinfo("PDF File", "File decryption Failed...")
    else:  
        messagebox.showinfo("PDF File", "File already decrypted.")


def convert(file):
    fname = file.split('/')[-1].split('.')[0]
    pages = convert_from_path(file, dpi = 500 , poppler_path=path)
    counter = 1
    for page in pages:
        myfile = fname + str(counter) +'.png'
        page.save(myfile)
#         print("File Saved at (path) ",myfile)
        counter = counter + 1
        messagebox.showinfo("PDF to IMG File", "Image File Saved Successfully as "+myfile)
    messagebox.showinfo("PDF to IMG File", "Files Saved Successfully")
        
def pdfToImg ():
    import_file_path = filedialog.askopenfilename()
    convert(import_file_path)
        
        
def autoCropImg():
    file = filedialog.askopenfilename()
    img = cv2.imread(file)
    # Starting Upper left Point x1, y1
    x1, y1 = 192, 192
    # End right Point x2,y2
    x2, y2 = 1889, 4349
    crop_img = img[y1:y1+(y2-y1), x1:x1+(x2-x1)]
    saveAsFileName = askstring(title="File Save As " , prompt="Enter File Name : ")
    saveAsFileName = saveAsFileName + '.png'
    cv2.imwrite(saveAsFileName,crop_img)
    messagebox.showinfo("IMG File", "Auto Crop Image File Saved Successfully as "+saveAsFileName)

def FinalAmask(file,opfilename):
    img = cv2.imread(file)
    img = cv2.resize(img, (1200, 2891))
#     cv2.imshow('Adhar Number with first 8 digit Mask', img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    text = pytesseract.image_to_string(threshold_img)
    data = pytesseract.image_to_data(threshold_img, output_type=pytesseract.Output.DICT)
    df = pd.DataFrame(data)  # Function Variable
    lines = text.split('\n')
    text1 = []
    for lin in lines:
        s = lin.strip()
        s = lin.replace('\n','')
        s = s.rstrip()
        s = s.lstrip()
        text1.append(s)
    text1 = list(filter(None, text1)) # Filter for removing Space as string like ' '
    # print("OCR Text List :\n",text1)
    # Using Regex find only number
    target_word = [a for a in text1 if re.search(re.compile("^[2-9]{1}[0-9]{3}\\s[0-9]{4}\\s[0-9]{4}$"), a)]
    #     print(target_word)
    def findCor(df,target_word):
        # Skip Last 4 digit and space
        SkipedNo=target_word[:-5]
        a, b = SkipedNo[:4], SkipedNo[-4:]
#         print("First 4 digit",a)
#         print("Second 4 digit",b)
    #     df = findAdharNumber.df # used variable from other function
        i = int(df[df['text']==a].index[0])
        j = int(df[df['text']==b].index[0])
        a, b = df['left'][i],df['top'][i]
        start_point = (a,b)
        c, d = df['left'][j]+df['width'][j], df['top'][j]+df['height'][j]
        end_point = (c,d)
    #     print(start_point, end_point)
        return start_point, end_point
    if len(target_word)>=1:
        for i in target_word:
            target_word = i
#             print("Aadhar Number : ",target_word)
            gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
            data = pytesseract.image_to_data(threshold_img, output_type=pytesseract.Output.DICT)
            df = pd.DataFrame(data)  # Function Variable
            start_point, end_point = findCor(df,target_word)
            image = cv2.rectangle(img, start_point, end_point, color = (0, 0, 0), thickness = 2)
#             cv2.imshow('Select first 8 digit of Aadhar Number', image)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()
            # show final image of Mask first 8 digit of Aadhar Number 
            img = cv2.rectangle(image, start_point, end_point, (0, 255, 0), cv2.FILLED)
#             cv2.imshow('Adhar Number with first 8 digit Mask', img)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()
        cv2.imwrite(opfilename,img)
    else:
        print("Not match any Aadhar Number")
        
def maskAadhar():
    file = filedialog.askopenfilename()
#     img = cv2.imread(file)
    saveAsFileName = askstring(title="File Save As " , prompt="Enter File Name : ")
    saveAsFileName = saveAsFileName + '.png'
    FinalAmask(file,saveAsFileName)
    messagebox.showinfo("IMG File", "Mask Aadhar Saved Successfully as "+saveAsFileName)
    
def exitApplication():
    MsgBox = tk.messagebox.askquestion ('Exit Application','Are you sure you want to exit the application', icon = 'warning')
    if MsgBox == 'yes':
        root.destroy()

browseButton0 = tk.Button(text="     Decrypt PDF File  ", command=decPdfFile, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas.create_window(150, 90, window=browseButton0)

browseButton0 = tk.Button(text="     PDF To IMG File  ", command=pdfToImg, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas.create_window(150, 130, window=browseButton0)

browseButton1 = tk.Button(text="  Auto Crop Aadhar IMG  ", command=autoCropImg, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas.create_window(150, 170, window=browseButton1)

browseButton1 = tk.Button(text="  Mask Adhar Number  ", command=maskAadhar, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas.create_window(150, 210, window=browseButton1)

exitButton = tk.Button (root, text='Exit Application',command=exitApplication, bg='brown', fg='white', font=('helvetica', 12, 'bold'))
canvas.create_window(150, 270, window=exitButton)

root.mainloop()

In [9]:
# Main Function 
def FinalAmask(file,opfilename):
    img = cv2.imread(file)
    img = cv2.resize(img, (1200, 2891))
    cv2.imshow('Adhar Number with first 8 digit Mask', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    text = pytesseract.image_to_string(threshold_img)
    data = pytesseract.image_to_data(threshold_img, output_type=pytesseract.Output.DICT)
    df = pd.DataFrame(data)  # Function Variable
    lines = text.split('\n')
    text1 = []
    for lin in lines:
        s = lin.strip()
        s = lin.replace('\n','')
        s = s.rstrip()
        s = s.lstrip()
        text1.append(s)
    text1 = list(filter(None, text1)) # Filter for removing Space as string like ' '
    # print("OCR Text List :\n",text1)
    # Using Regex find only number
    target_word = [a for a in text1 if re.search(re.compile("^[2-9]{1}[0-9]{3}\\s[0-9]{4}\\s[0-9]{4}$"), a)]
    #     print(target_word)
    def findCor(df,target_word):
        # Skip Last 4 digit and space
        SkipedNo=target_word[:-5]
        a, b = SkipedNo[:4], SkipedNo[-4:]
        print("First 4 digit",a)
        print("Second 4 digit",b)
    #     df = findAdharNumber.df # used variable from other function
        i = int(df[df['text']==a].index[0])
        j = int(df[df['text']==b].index[0])
        a, b = df['left'][i],df['top'][i]
        start_point = (a,b)
        c, d = df['left'][j]+df['width'][j], df['top'][j]+df['height'][j]
        end_point = (c,d)
    #     print(start_point, end_point)
        return start_point, end_point

    if len(target_word)>=1:
        for i in target_word:
            target_word = i
            print("Aadhar Number : ",target_word)
            gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
            data = pytesseract.image_to_data(threshold_img, output_type=pytesseract.Output.DICT)
            df = pd.DataFrame(data)  # Function Variable
            start_point, end_point = findCor(df,target_word)
            image = cv2.rectangle(img, start_point, end_point, color = (0, 0, 0), thickness = 2)
            cv2.imshow('Select first 8 digit of Aadhar Number', image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            # show final image of Mask first 8 digit of Aadhar Number 
            img = cv2.rectangle(image, start_point, end_point, (0, 255, 0), cv2.FILLED)
            cv2.imshow('Adhar Number with first 8 digit Mask', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        cv2.imwrite(opfilename,img)
    else:
        print("Not match any Aadhar Number")
    
# opfilename = "fgs11.png"
# file = "as111.png"
# FinalAmask(file,opfilename)